In [ ]:
import pandas as pd
import numpy as np
import os

from gensim.models import Word2Vec

from classes import *

%run functions.py

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler


In [ ]:
# if data not saved as csv, run this
import os
if not os.path.exists('../data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('../data/merged_titles_labels.csv')
df.head()


In [ ]:
%run functions.py
if not os.path.exists('../data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df, verbose=True)
    df.to_pickle('../data/preprocessed_titles_labels.pkl') 

else:
    df = pd.read_pickle('../data/preprocessed_titles_labels.pkl')
df.head()

In [ ]:
# read results from previous runs json file
save_path = 'results/word2vec_results_10_20_50_100.json'
import json

# if file doesn't exist, create it
if not os.path.exists(save_path):
    with open(save_path, 'w') as f:
        json.dump({}, f)
        
with open(save_path, 'r') as f:
    word2vec_results = json.load(f)


# get all the models from the results
models_already_trained = []
for model in word2vec_results:
    models_already_trained.append(model)
models_already_trained = [model + ".model" for model in models_already_trained]
models_already_trained

In [ ]:
from tqdm import tqdm
import copy as cp


files = os.listdir('../word2vec_models')
files = [file for file in files if file.endswith('.model')]


files10 = [file for file in files if file.startswith('word2vec_vs10_')]
files20 = [file for file in files if file.startswith('word2vec_vs20_')]
files50 = [file for file in files if file.startswith('word2vec_vs50_')]
files100 = [file for file in files if file.startswith('word2vec_vs100_')]


files = files10 + files20 + files50 + files100  

# remove the files that have already been trained
files = [file for file in files if file not in models_already_trained]

print(files)


In [ ]:
import copy as cp
from tqdm.auto import tqdm
# iterate over files in word2vec_models folder
for file in tqdm(files):
    # check if file is a .model file
    if file.endswith('.model'):
        df_temp = cp.deepcopy(df)
        # load model
        # path to model
        model_path = os.path.join('../word2vec_models', file)

        properties = file.split('_')
        # print(properties)

        vector_size = int(properties[1][2:])
        window_size = int(properties[2][3:])
        is_skipgram = bool(int(properties[3][2:-6]))

        settings = {
            'model_path': model_path,
            'is_skipgram' : is_skipgram,
            'window_size' : window_size,
            'vector_size' : vector_size, 

        }
        # print(settings)

        model_w2v = Word2VecModel(settings)
        # get model name
        model_name = file.split('.')[0]

        title_vectors = [get_word_vectors(model_w2v, title, aggregation='mean') for title in df_temp['title']]
        df_temp['title_vector'] = title_vectors

        train = df_temp['sample'] == 'train'
        test = df_temp['sample'] == 'test'

        # X = np.vstack(title_vectors)
        # y = df['is_clickbait'].values

        X_train = np.vstack(df_temp[train]['title_vector'])
        y_train = df_temp[train]['is_clickbait'].values

        X_test = np.vstack(df_temp[test]['title_vector'])
        y_test = df_temp[test]['is_clickbait'].values
   

        # scale data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # train models
        models = {
            'catboost': [
                CatBoostClassifier(verbose=False, random_state=1, ),
                CatBoostClassifier(verbose=False, random_state=2, ),
                CatBoostClassifier(verbose=False, random_state=3, ),
             
            ],

            'lightgbm': [
                LGBMClassifier(random_state=1, verbose=-1),
                LGBMClassifier(random_state=2, verbose=-1),
                LGBMClassifier(random_state=3 ,verbose=-1),
            
            ],
            
            'knn' : [
                KNeighborsClassifier(n_neighbors=3),
                KNeighborsClassifier(n_neighbors=4),
                KNeighborsClassifier(n_neighbors=5),
               
            ],

            'randomforest': [
                RandomForestClassifier(random_state=1,),
                RandomForestClassifier(random_state=2, ),
                RandomForestClassifier(random_state=3, ),
        

            ],

            'decisiontree': [
                DecisionTreeClassifier(random_state=1, ),
                DecisionTreeClassifier(random_state=2, ),
                DecisionTreeClassifier(random_state=3, ),
               
            ],
        }


        results = {}

        for model_type in tqdm(models, desc=f'Predictive Models for {model_name}'):
            f1_temp = list()
            auc_temp = list()
            # print(model_type)
            for submodel in tqdm(models[model_type], desc=f'{model_type} submodels'):
                submodel.fit(X_train_scaled, y_train)
                y_pred = submodel.predict(X_test_scaled)
                f1 = f1_score(y_test, y_pred)
                auc = roc_auc_score(y_test, y_pred)
                f1_temp.append(f1)
                auc_temp.append(auc)
                
            results[model_type] = {
                'f1': np.mean(f1_temp),
                'auc': np.mean(auc_temp),
                'f1_list': cp.deepcopy(f1_temp),
                'auc_list': cp.deepcopy(auc_temp),
            }
            
            
        word2vec_results[model_name] = results
        # save results to json
        with open(save_path, 'w') as f:
            json.dump(word2vec_results, f, indent=4)
        print(f'{model_name} done and updated json')



